### EMA imports

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from ema_workbench import (ReplicatorModel, RealParameter, BooleanParameter, IntegerParameter, 
Constant, TimeSeriesOutcome, perform_experiments, save_results, ema_logging)

ema_logging.log_to_stderr(ema_logging.INFO)

### Define Mesa model

In [ ]:
def model_virus_on_network(num_nodes=1, 
                            avg_node_degree=1, 
                            initial_outbreak_size=1, 
                            virus_spread_chance=1, 
                            virus_check_frequency=1, 
                            recovery_chance=1, 
                            gain_resistance_chance=1,
                            steps=10):
    
    from virus_on_network import model
    
    virus_on_network = model.VirusOnNetwork(num_nodes=num_nodes, 
                                avg_node_degree=avg_node_degree, 
                                initial_outbreak_size=initial_outbreak_size, 
                                virus_spread_chance=virus_spread_chance, 
                                virus_check_frequency=virus_check_frequency, 
                                recovery_chance=recovery_chance, 
                                gain_resistance_chance=gain_resistance_chance)
    
    virus_on_network.run_model(steps)
    outcomes = virus_on_network.datacollector.get_model_vars_dataframe()
    
    return {"Infected": outcomes["Infected"].tolist(),
            "Susceptible": outcomes["Susceptible"].tolist(),
            "Resistant": outcomes["Resistant"].tolist()}
    

### Test Mesa model

In [ ]:
model_virus_on_network(num_nodes=20, 
                            avg_node_degree=3, 
                            initial_outbreak_size=1, 
                            virus_spread_chance=0.4, 
                            virus_check_frequency=0.2, 
                            recovery_chance=0.4, 
                            gain_resistance_chance=0.3,
                            steps=10)

### Design EMA Experiments

In [ ]:
model = ReplicatorModel('virusnetwork', function=model_virus_on_network)

In [ ]:
model.uncertainties = [IntegerParameter("num_nodes", 10, 100),
                       IntegerParameter("avg_node_degree", 2, 8),
                        RealParameter("virus_spread_chance", 0.1, 1),
                        RealParameter("virus_check_frequency", 0.1, 1),
                        RealParameter("recovery_chance", 0.1, 1),
                        RealParameter("gain_resistance_chance", 0.1, 1)]

In [ ]:
model.constants = [Constant("initial_outbreak_size", 1),
                  Constant('steps', 10)]

In [ ]:
model.outcomes = [TimeSeriesOutcome('Infected'),
                 TimeSeriesOutcome('Susceptible'),
                 TimeSeriesOutcome('Resistant')]

In [ ]:
model.replications = 10

In [ ]:
results = perform_experiments(model, 100)

experiments, outcomes = results

In [ ]:
outcomes['Infected']

### Process outcomes

In [ ]:
# average over replications

### Visualize outcomes

In [ ]:
# matplotlib stuff